In [7]:
%pip install alpha_vantage python-dotenv plotly

Note: you may need to restart the kernel to use updated packages.


In [22]:
from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import pandas as pd
import os

load_dotenv()

# 設定你的 Alpha Vantage API 金鑰
api_key = os.getenv("ALPHA_VINTAGE_API_KEY")

# 初始化 TimeSeries 物件
ts = TimeSeries(key=api_key, output_format='pandas')


In [23]:
def get_stock_data(symbol, interval, months):
    """
    獲取指定股票的資料。
    
    參數:
    - symbol: 股票代碼，如 'AAPL'。
    - interval: 數據週期，如 '1min'、'60min'、'daily'。
    - months: 時間長度（月），如 1 (一個月)、3 (三個月)、6 (半年)、12 (一年)。
    
    返回:
    - Pandas DataFrame: 股票資料。
    """
    
    # 根據選擇的月份調整 outputsize
    if months <= 3:
        outputsize = 'compact'
    else:
        outputsize = 'full'
    
    if interval == 'daily':
        data, meta_data = ts.get_daily(symbol=symbol, outputsize=outputsize)
    elif interval.endswith('min'):
        data, meta_data = ts.get_intraday(symbol=symbol, interval=interval, outputsize=outputsize)
    else:
        raise ValueError("Unsupported interval. Choose '1min', '60min', or 'daily'.")
    
    # 篩選指定時間範圍內的數據
    end_date = pd.Timestamp.now()
    start_date = end_date - pd.DateOffset(months=months)
    data_filtered = data[(data.index >= start_date) & (data.index <= end_date)]
    
    return data_filtered

In [29]:
symbol = 'AAPL'
interval = 'daily'
months = 3

data = get_stock_data(symbol, interval, months)
data.head()

,1. open,2. high,3. low,4. close,5. volume
date,,,,,
2024-03-15,171.17,172.6200,170.285,172.62,121752699.0
2024-03-14,172.91,174.3078,172.050,173.00,72571635.0
2024-03-13,172.77,173.1850,170.760,171.13,51948951.0
2024-03-12,173.15,174.0300,171.010,173.23,59544927.0
2024-03-11,172.94,174.3800,172.050,172.75,58929918.0


In [30]:
# change index name to time
data.index.name = 'time'

In [31]:
data.columns = ['open', 'high', 'low', 'close', 'volume']

In [27]:
data.head()

,open,high,low,close,volume
time,,,,,
2024-03-15 19:45:00,172.640,172.70,172.630,172.63,9309.0
2024-03-15 19:30:00,172.640,172.73,172.630,172.63,3833.0
2024-03-15 19:15:00,172.611,172.75,172.565,172.73,8400.0
2024-03-15 19:00:00,172.610,172.74,172.450,172.61,16042.0
2024-03-15 18:45:00,172.390,172.62,172.320,172.60,21724.0


In [15]:
import pandas as pd
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=data.index, y=data['close'], mode='lines', name=symbol))
fig.update_layout(title=f'{symbol} Stock Price (Last {months} Months)',
                    xaxis_title='Date',
                    yaxis_title='Price (USD)',
                    template='plotly_dark')
fig.show()

In [32]:
import plotly.graph_objects as go

# 繪製蠟燭圖
fig = go.Figure(data=[go.Candlestick(x=data.index,
                open=data['open'],
                high=data['high'],
                low=data['low'],
                close=data['close'])])

# 更新圖表布局
fig.update_layout(title='Stock Price Candlestick Chart',
                  xaxis_title='Date',
                  yaxis_title='Price (USD)',
                  xaxis_rangeslider_visible=False)  # 隱藏範圍滑塊

fig.show()

In [11]:
from binance.client import Client


client = Client(os.getenv("BINANCE_API_KEY"), os.getenv("BINANCE_SECRET_KEY"))
client.API_URL = "https://testnet.binance.vision/api"

In [12]:
ticker = "BTCUSDT"
timeframe = "1d"

timestamp = client._get_earliest_valid_timestamp(ticker, timeframe)
bars = client.get_historical_klines(ticker, timeframe, timestamp)
for line in bars:
    del line[6:]

btc_df = pd.DataFrame(
    bars, columns=["time", "open", "high", "low", "close", "volume"]
)

In [13]:
btc_df.head()

,time,open,high,low,close,volume
0,1710288000000,72705.81000000,80000.00000000,14473.00000000,73083.96000000,114.63568000
1,1710374400000,73083.96000000,80000.00000000,30000.00000000,71405.34000000,385.75314000
2,1710460800000,71405.34000000,72287.00000000,20000.00000000,69374.89000000,335.30817000
3,1710547200000,69381.11000000,70600.00000000,63695.00000000,69020.01000000,52.20057000
